# KPMG Virtual Data Analytics Internship

## Problem scenerio

For context, Sprocket Central Pty Ltd is a long-standing KPMG client whom specialises in high-quality bikes and accessible cycling accessories to riders. Their marketing team is looking to boost business by analysing their existing customer dataset to determine customer trends and behaviour. 

Using the existing 3 datasets (Customer demographic, customer address and transactions) as a labelled dataset, please recommend which of these 1000 new customers should be targeted to drive the most value for the organisation. 

In building this recommendation, we need to start with a PowerPoint presentation which outlines the approach which we will be taking. The client has agreed on a 3 week scope with the following 3 phases as follows - Data Exploration; Model Development and Interpretation.

Prepare a detailed approach for completing the analysis including activities – i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off. Please advise what steps you would take. 

Please ensure your PowerPoint presentation includes a detailed approach for our strategy behind each of the 3 phases including activities involved in each - i.e. understanding the data distributions, feature engineering, data transformations, modelling, results interpretation and reporting. This detailed plan needs to be presented to the client to get a sign-off.



### Task 2:  Data Quality Assessment for Existing Customers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings 
warnings.filterwarnings("ignore")

# Merge the dataset within one sheet

# First read the data set into a seperate dataframes 
demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerDemographic',skiprows=1)
cust_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='CustomerAddress',skiprows=1)
transanction = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='Transactions',skiprows=1)
new_cust_list = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name='NewCustomerList',skiprows=1)

# Merge the dataset in one dataframe for further analysis
df= pd.merge(demographic, transanction, on = 'customer_id')
df= pd.merge(df,cust_address, on = 'customer_id')
df

### Data Exploration 

In [ ]:
# Dataframe dimension (rows, columns)

df.shape

In [ ]:
# Data Profiling (i.e. data types, non null values and columns)

df.info()

In [ ]:
# Checking any duplicate values in the datset

df.duplicated().sum()  # No duplicate values

In [ ]:
# Null values in the datset

df.isnull().sum()

In the demographic sheet their are multiple column having missing dataset i.e last name, DOB, job title, job_industry_category,  job_industry_category,  default, tenure,  online_order,  product_class and product_size etc.

In [ ]:
# Check unique values in each columns;

df.nunique()

In [ ]:
# Explore the columns

df.columns

In [ ]:
# Checking Gender categories

df['gender'].value_counts()

In [ ]:
# Replace the categorical values

df['gender'].replace('M', 'Male', inplace=True)
df['gender'].replace('F', 'Female', inplace=True)
df['gender'].replace('Femal', 'Female', inplace=True)
df['gender'].replace('U', 'Unspecified', inplace=True)

gender_counts = df['gender'].value_counts()

# Plotting the gender counts with color
plt.bar(gender_counts.index, gender_counts.values, color=['pink', 'lightgreen', 'lavender'])
plt.xlabel('Gender')
plt.ylabel('Count')
plt.title('Gender Distribution')

plt.show()


In [ ]:
# Checking DOB column for any ages above 100

# Get the current date
current_date = datetime.now()       

# Calculate the age by subtracting the DOB from the current date
df['age'] = (current_date - df['DOB']) / pd.Timedelta(days=365.25)

# Filter rows where age is greater than 100
age_greater_than_100 = df[df['age'] > 100]

# print age greater than 100
age_greater_than_100

Here, we have one customers with customer id =34, having age more than 100, which is practically impossible. Thus, we need drop it.

In [ ]:
# Drop rows where customer ID = 34
customer_id = 34

# Drop rows with customer ID 34
df = df.drop(df[df['customer_id'] == customer_id].index)

In [ ]:
# Checking job_title 

df['job_title'].value_counts()


It seems like more of the individula work as a Social Worker, Business analyst, Assistant Professor,Legal Assistant, Executive secretary are some of the prime users.

In [ ]:
# Checking top customers job category
job_industry_category = df['job_industry_category'].value_counts()

# Sort the counts in descending order
job_industry_category = job_industry_category.sort_values(ascending=False)

# Plotting the Job Industry counts with color using Seaborn
plt.figure(figsize=(10, 6))
sns.countplot(x='job_industry_category', data=df, order=job_industry_category.index, palette='Set3')
plt.xlabel('Job Industry')
plt.ylabel('Count')
plt.title('Job Industry Distribution')

# Rotate x-axis tick labels
plt.xticks(rotation=45, ha='right')

plt.show()

It seems like more of the individula work for Manufacturing, Financial Services & Health are the prime users followed by Retail, Property & IT domain. 

In [ ]:
# Checking wealth_segment
wealth_segment = df['wealth_segment'].value_counts()

# Plotting the wealth segment counts with hue
sns.countplot(x='wealth_segment', data=df, hue='gender', palette='Set3')
plt.xlabel('Wealth Segment')
plt.ylabel('Count')
plt.title('Wealth Segment Distribution')

plt.show()


In [ ]:
# Dropping default column

df['default'].value_counts()
df = df.drop('default', axis=1)

This default column is of no importance and can be deleted.

In [ ]:
# Checking deceased_indicator 

df['deceased_indicator'].value_counts()

We have two deceased customer

In [ ]:
# Checking owns_car 

df['owns_car'].value_counts()

Looks like we have equal number of customers who has a car and who doesnt have a car.

In [ ]:
# Checking tenure 

df['tenure'].value_counts()


In [ ]:
df['product_line'].value_counts()

In [ ]:
# Checking Product class

df['product_class'].value_counts()

In [ ]:
# Grouping data by product line and product class and getting their counts
product_line_class_counts = df.groupby(['product_line', 'product_class']).size().unstack().fillna(0)

# Sort the counts in descending order
product_line_class_counts = product_line_class_counts.sort_values(by=product_line_class_counts.columns.tolist(), ascending=False)

# Plotting the bar chart
plt.figure(figsize=(10, 6))
product_line_class_counts.plot(kind='bar', stacked=True)
plt.xlabel('Product Line')
plt.ylabel('Count')
plt.title('Product Line and Class Distribution')
plt.legend(title='Product Class')

plt.show()


In [ ]:
# Checking Top 10 Product ID
top_10_product_id = df['product_id'].value_counts().nlargest(10)

# Plotting the top 10 Product ID counts using Seaborn
plt.figure(figsize=(8, 6))
plt.pie(top_10_product_id.values, labels=top_10_product_id.index, autopct='%1.1f%%', startangle=90)
plt.title('Top 10 Product (id) Sale')

plt.show()


In [ ]:
# Checking States
df['state'].value_counts()

In [ ]:
# Replace the categorical values
df['state'].replace('New South Wales', 'NSW', inplace=True)
df['state'].replace('Victoria', 'VIC', inplace=True)

state = df['state'].value_counts()

# Plotting the gender counts with color
plt.bar(state.index, state.values, color=['pink', 'lightgreen', 'lavender'])
plt.xlabel('state')
plt.ylabel('Count')
plt.title('State Distribution')

plt.show()


In [ ]:
# Visualize the age groups

# Binning the age values
bin_edges = range(0, 100, 10)  # Define the bin edges (0, 10, 20, ..., 100)
plt.hist(df['age'], bins=bin_edges, edgecolor='black')

# Set labels and title
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Customer Age Distribution')

plt.show()

Most of the customer age group are 40-50

In [ ]:
# Visualize the Property valuation

# Binning the age values
bin_edges = range(0, 15, 1) 
plt.hist(df['property_valuation'], bins=bin_edges, edgecolor='black')

# Set labels and title
plt.xlabel('property_valuation')
plt.ylabel('Count')
plt.title('Property valuation distribution')

Respective customers having a property valuation range from 7.5 to 10.5 are mostly using the product

In [ ]:
# looking for distribution
nums = ['past_3_years_bike_related_purchases', 'tenure', 'age', 'property_valuation']
plt.figure(figsize=(15, 3))
for i in range(0, len(nums)):
    plt.subplot(1, 5, i+1)
    sns.distplot(df[nums[i]])
    plt.tight_layout()

In [ ]:
# Transaction analysis: 

# making backup dataset
df_transact = df.copy()

# deeper analysis of datetime data by breaking it down into more specific parts
df_transact['transaction_year'] = df['transaction_date'].dt.strftime('%Y')
df_transact['transaction_month'] = df['transaction_date'].dt.strftime('%B')
df_transact['transaction_quarter'] = df['transaction_date'].dt.quarter.astype(str)
df_transact['transaction_day'] = df['transaction_date'].dt.strftime('%d')
df_transact['transaction_weekday'] = df['transaction_date'].dt.strftime('%A')
df_transact.drop(['transaction_date'],axis=1,inplace=True)

# aggregate by month
transaction_per_month = df_transact.groupby(['transaction_month'])[['customer_id']].count()
transaction_per_month.reset_index(inplace=True)
transaction_per_month.columns = ['month', 'number_of_transaction']

# order by month
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
transaction_per_month['month'] = pd.Categorical(transaction_per_month['month'], month_order)
transaction_per_month.sort_values('month', inplace=True)
transaction_per_month


In [ ]:
# visualization transanctions overtime.
fig = plt.figure(figsize=(20, 5))

fig.suptitle('No\'s of Transction per month', fontsize=20, fontweight='bold')
plt.xlabel('Month', fontsize=14)
plt.ylabel('Number of Transaction', fontsize=14)

x = transaction_per_month['month']
y = transaction_per_month['number_of_transaction']
mean = y.mean()

plt.plot(x, y, color = 'steelblue')
plt.grid(color='darkgray', linestyle=':', linewidth=0.5)
plt.axhline(y=mean, color='red', linestyle=':')
plt.text(0.25, 0.5, 'Mean : No\'s of Transction', transform=fig.transFigure, color='red')
plt.show()

In [ ]:
# Checking product details

stat = df[['brand', 'product_line', 'product_class', 'product_size']].describe().T
stat['% top'] = stat['freq']/stat['count']*100
stat

Customers predominantly purchased Solex, accounting for 21.43% of all purchases.

The majority of customers, comprising 71.59%, opted for Standard bikes.

A significant proportion of customers, accounting for 69.84%, purchased medium class bikes.

Approximately 65.68% of customers opted for medium size bikes.

In [ ]:
# Brand Analysis
brand = df['brand'].value_counts().reset_index()
brand.columns = ['brand', 'count']
brand['percentage'] = (brand['count']/brand['count'].sum()*100).round(2)
brand

In [ ]:
# visualizing brand 

fig, ax = plt.subplots(figsize=(10, 5))

# Sorting the data by percentage in ascending order
brand_sort = brand.sort_values(by=['percentage'])

# Plotting the pie chart
wedges, texts, autotexts = ax.pie(brand_sort['percentage'], labels=brand_sort['brand'], autopct='%1.1f%%', startangle=90)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
ax.set_title('Percentage of Brand', fontsize=16, fontweight='bold')

# Adding percentage labels to the pie chart
for autotext in autotexts:
    autotext.set_color('white')  # Set label color to white

plt.show()


In [ ]:
# Export the excel for dashboard visualization

df.to_excel('Dashboard data.xlsx', index=False)

### Conclusion:

Based on the above analysis, the following key findings and recommendations can be made:

1. Industry Preferences: The major user segments are observed in the Manufacturing, Financial Services, and Health industries, followed by Retail, Property, and IT domains. To optimize business growth, it is recommended to prioritize these industries in marketing and sales strategies.

2. Targeted Professions: The prime users of the product include individuals working as Social Workers, Business Analysts, Assistant Professors, Legal Assistants, and Executive Secretaries. To capitalize on this trend, it is recommended to focus marketing efforts towards these professional segments.

3. Popular Product Line: The Standard product line exhibits the highest sales with a count of 14,151. This indicates its popularity among customers. To leverage this demand, it is advised to ensure sufficient availability and promotion of Standard products.

4. Best-selling Products: The most sold products are identified as Product IDs O, 3, and 1. It is recommended to monitor and maintain adequate stock levels of these products to meet customer demand.

5. Geographic Focus: The majority of customers are located in the state of NSW. To maximize market penetration and customer engagement, targeted marketing campaigns and promotional activities should be concentrated in this region.

6. Target Age Group: The analysis reveals that customers in the age group of 40-50 are the primary users of the product. Tailored marketing strategies and product offerings should cater to the preferences and needs of this age group to attract and retain customers.

7. Property Valuation: Customers with property valuations ranging between 7-10.5 demonstrate a higher propensity for purchasing the product. This segment should be targeted with customized offers and incentives to drive sales.

8. Brand Performance: The brand with the highest transaction percentage is Solex at 21.48%, followed by Giant Bicycles (16.73%) and WeareA2B (16.63%). To capitalize on the popularity of these brands, marketing efforts should emphasize their unique selling propositions and strengthen brand loyalty.

In conclusion, by considering these recommendations and aligning marketing strategies accordingly, the overall business growth and customer acquisition can be optimized in a more professional and targeted manner.